In [1]:
%matplotlib widget

In [2]:
from bmcs_matmod.slide.slide_explorer import SlideExplorer
import bmcs_utils.api as bu

Parameters for elementary convergence studies - no physical relevance

In [3]:
material_params = dict(
    E_s=1, gamma_s=5, K_s=3, S_s=0.6, c_s=1, bartau=1, 
    E_w=1, S_w=0.6, c_w = 1, m = 0.01, f_t=1, f_c=-12, f_c0=-8, eta=0.5
)

material_params = dict(
    E_s=1, gamma_s=5, K_s=5, S_s=0.6, c_s=1, bartau=1, 
    E_w=1, S_w=0.6, c_w = 1, m = 0.01, f_t=1, f_c=-20, f_c0=-10, eta=0
)

Parameters rendering a stress-strain response comparable with a `MARS` model - example concrete

In [4]:
E_w = 28000
alpha = 0.9
E_s = alpha * E_w
material_params = dict(
    E_s=E_s, gamma_s=0, K_s=0, S_s=0.01, c_s=1, bartau=4, 
    E_w=E_w, S_w=0.0001, c_w = 1, m = 0.01, f_t=4, f_c=-100, f_c0=-20, eta=0
)

## Debugging LDM implementation

In [5]:
material_params = dict(
    E_s=1, gamma_s=1, K_s=1, S_s=1, c_s=1, bartau=1, 
    E_w=1, S_w=0.5, c_w = 1, m = 0.05, f_t=1, f_c=-20, f_c0=-10, eta=0.1
)

Critical step size - step smaller than the elastic range in tension.

In [6]:
def get_step_cr(**kw):
    f_t = kw['f_t']
    E_w = kw['E_w']
    return f_t/E_w
d_w = get_step_cr(**material_params)
w_max = 10
n_steps = int(w_max / d_w)
n_steps

10

In [11]:
se = SlideExplorer(n_steps=1000, k_max=50)
se.slide_model.trait_set(**material_params)
se.trait_set(s_x_1 = 2, s_y_1 = 0, w_1 = 0, r_tol=0.1);

In [12]:
bu.InteractiveWindow([se, se.energy_dissipation, se.inel_state_evolution, 
                      se.slide_model], figsize=(9,4), tight_layout=False).interact()

<lambdifygenerated-18>:4: RuntimeWarning: invalid value encountered in double_scalars
  return (select([less((abs(sqrt((-X_x + tau_pi_x/(1 - omega_s))**2 + (-X_y + tau_pi_y/(1 - omega_s))**2)) - (Z + bartau)*(sigma_pi/(1 - omega_w) + f_t**2*m/(Z + bartau - 2*f_t*m))*(Z + bartau - 2*f_t*m)/(f_t**2*m))*sign(f_t)*sign(m), 0),less((abs(sqrt((-X_x + tau_pi_x/(1 - omega_s))**2 + (-X_y + tau_pi_y/(1 - omega_s))**2)) - (Z + bartau + f_c0*m)*(sigma_pi/(1 - omega_w) + f_c0 + m*(f_c**2 - 2*f_c*f_c0 + f_c0**2)/(Z + bartau + 2*f_c*m - f_c0*m))*(Z + bartau + 2*f_c*m - f_c0*m)/(m*(f_c**2 - 2*f_c*f_c0 + f_c0**2)))*sign(m)*sign(-f_c + f_c0), 0),True], [sqrt(((-X_x + tau_pi_x/(1 - omega_s))**2 + (-X_y + tau_pi_y/(1 - omega_s))**2)*(f_t**2*m**2 - 2*f_t*m*(Z + bartau) + (Z + bartau)**2)**2/((Z + bartau)*(-Z - bartau + f_t*m)**2*(Z + bartau - 2*f_t*m)) + (sigma_pi/(1 - omega_w) + f_t**2*m/(Z + bartau - 2*f_t*m))**2*(f_t**2*m**2 - 2*f_t*m*(Z + bartau) + (Z + bartau)**2)**2/(f_t**2*(Z + bartau - f_t*m)**2)) 

<lambdifygenerated-18>:4: RuntimeWarning: invalid value encountered in sqrt
  return (select([less((abs(sqrt((-X_x + tau_pi_x/(1 - omega_s))**2 + (-X_y + tau_pi_y/(1 - omega_s))**2)) - (Z + bartau)*(sigma_pi/(1 - omega_w) + f_t**2*m/(Z + bartau - 2*f_t*m))*(Z + bartau - 2*f_t*m)/(f_t**2*m))*sign(f_t)*sign(m), 0),less((abs(sqrt((-X_x + tau_pi_x/(1 - omega_s))**2 + (-X_y + tau_pi_y/(1 - omega_s))**2)) - (Z + bartau + f_c0*m)*(sigma_pi/(1 - omega_w) + f_c0 + m*(f_c**2 - 2*f_c*f_c0 + f_c0**2)/(Z + bartau + 2*f_c*m - f_c0*m))*(Z + bartau + 2*f_c*m - f_c0*m)/(m*(f_c**2 - 2*f_c*f_c0 + f_c0**2)))*sign(m)*sign(-f_c + f_c0), 0),True], [sqrt(((-X_x + tau_pi_x/(1 - omega_s))**2 + (-X_y + tau_pi_y/(1 - omega_s))**2)*(f_t**2*m**2 - 2*f_t*m*(Z + bartau) + (Z + bartau)**2)**2/((Z + bartau)*(-Z - bartau + f_t*m)**2*(Z + bartau - 2*f_t*m)) + (sigma_pi/(1 - omega_w) + f_t**2*m/(Z + bartau - 2*f_t*m))**2*(f_t**2*m**2 - 2*f_t*m*(Z + bartau) + (Z + bartau)**2)**2/(f_t**2*(Z + bartau - f_t*m)**2)) - (f_t**2*

In [13]:
se.Sig_arr[-1,:]

array([ 9.46480508e-01,  4.41381278e-10, -1.46598967e-02,  3.05813810e-01,
        3.05813810e-01,  2.03294239e-10,  1.29994502e+00,  1.24767791e-04])

In [37]:
se.Sig_arr[-1,:]

array([ 9.46480508e-01,  4.41381278e-10, -1.46598967e-02,  3.05813810e-01,
        3.05813810e-01,  2.03294239e-10,  1.29994502e+00,  1.24767791e-04])

In [ ]:
material_params = dict(
    E_s=1, gamma_s=5, K_s=5, S_s=0.6, c_s=1, bartau=1,
    E_w=1, S_w=0.6, c_w=1, m=0.01, f_t=1, f_c=-20, f_c0=-10, eta=0.5
)

material_params = dict(
    E_s=1, gamma_s=5, K_s=5, S_s=0.6, c_s=1, bartau=1,
    E_w=1, S_w=0.6, c_w=1, m=0.01, f_t=1, f_c=-20, f_c0=-10, eta=1
)

material_params = dict(
    E_s=1, gamma_s=0, K_s=-0.1, S_s=10000, c_s=1, bartau=1,
    E_w=1, S_w=1000, c_w=1, m=0.01, f_t=1, f_c=-20, f_c0=-10, eta=1
)


material_params = dict(
    E_s=1, gamma_s=-0.1, K_s=0, S_s=10000, c_s=1, bartau=1,
    E_w=1, S_w=100000, c_w=1, m=0.00001, f_t=100, f_c=-20000, f_c0=-10000, eta=0
)